In [1]:
import requests as rq
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML

In [2]:
cases_endpt = 'https://api.gdc.cancer.gov/cases'

# query file

See https://docs.gdc.cancer.gov/API/Users_Guide/Appendix_A_Available_Fields/#file-fields for possible *fields*

In [ ]:
fields = [
    "submitter_id",
    "case_id",
    "primary_site",
    "disease_type",
    "diagnoses.tumor_stage",
    "diagnoses.tumor_grade",
    "diagnoses.primary_diagnosis",
    "diagnoses.classification_of_tumor",
    "annotations.classification",
    "samples.tumor_code"
    ]

fields = ','.join(fields)

In [ ]:
def queryFile(idFile):
    filters = {
    "op": "in",
    "content":{
        "field": "files.file_name",
        "value": [idFile]
        }
    }
    params = {
    "fields": fields,
    "filters": json.dumps(filters),
    "format": "TSV",
    "size": "1"
    }
    print("quering...%s"%idFile)
    response = rq.get(cases_endpt, params = params)
    #print(response.content.decode('utf-8'))
    r = response.content.decode("utf-8").split('\r')
    data = np.array(r[1].replace('\n','').split('\t'))
    data = data.reshape(1,len(data))
    return pd.DataFrame(data=data, columns=r[0].split('\t'), index=[0])

In [ ]:
queryFile('418eab89-5e78-4be3-ba93-c9dc2a7d1c68.FPKM.txt.gz')

## query many

In [ ]:
files = []

In [5]:
cluster = []

In [ ]:
def queryFiles(files):
    df = pd.DataFrame(columns=fields.split(','))
    for i,f  in enumerate(files):
        df = df.append(queryFile(f), ignore_index=True)
    print(df.head())
    return df

In [ ]:
def makePie(df, l, c, col = 'disease_type'):
    col_counts = df.loc[:,col].value_counts()
    fig = plt.figure(figsize=(15,8))
    col_counts.plot(kind='pie', fontsize=16, title = col)
    fig.savefig("cluster_pie_%s_l_%d_c%d.pdf"%(col,l, c))

In [ ]:
for c in cluster:
    makePie(queryFiles([f[0][:-2]+".FPKM.txt.gz" for f in cluster[c]]), 0, c, 'primary_site')

# Words
## query words

In [1]:
cluster = [('3', 1.0), ('65', 1.0), ('89', 1.0), ('9', 1.0), ('34', 1.0)]

In [ ]:
for c,_ in cluster:
    print(df.keys()[int(c)][:-2])